# 14.75 PS2 (Max Ghenis)

## (1)

Use `mitaData.RData` for the following exercise.

In [1]:
suppressMessages({
    library(dplyr)
    library(magrittr)
    library(multiwayvcov)
    library(lmtest)
})

Warning message:
“package ‘zoo’ was built under R version 3.6.2”

In [2]:
load('mitaData.RData')

### (a)
>Generate given longitude and latitude variables x, y, construct x2, y2, xy, x3, y3, x2y, xy2.

In [3]:
mitaData %<>% mutate(x2 = x^2,
                     y2 = y^2,
                     xy = x*y,
                     x3 = x^3,
                     y3 = y^3,
                     x2y = y*x^2,
                     xy2 = x*y^2)

### (b)

> Regress the log equivalent household consumption (2001) [lhhequiv] on mita [pothuanmita], all polynomial terms in (a), elevation [elv-sh], mean slope [slope], infants, children, adults [variable names the same], and boundary segment fixed effects [bfe4-1,
bfe4-2, bfe4-3]. Cluster the standard errors by district [district]. Run the regression
in 3 ways: first, for observations where the distance to mita boundary [d-bnd] is less
than 100km, next when it is less than 75km, and lastly when it is less than 50km.

In [4]:
names(mitaData)

[1] "conglome"     "vivienda"     "hogar"        "district"     "male"        
 [6] "age"          "ccdd"         "ccpp"         "ccdi"         "hid"         
[11] "hhmem"        "lhhmem"       "kids"         "k_hhmem"      "infants"     
[16] "children"     "adults"       "ces"          "depprov"      "hconsump"    
[21] "hconsumplm"   "hhequiv"      "lhhequiv"     "lhhconsplm"   "CAST"        
[26] "QUE"          "AYM"          "near_x"       "near_y"       "cusco"       
[31] "pothuan_mita" "border"       "d_bnd"        "bfe4_1"       "bfe4_2"      
[36] "bfe4_3"       "elv_sh"       "slope"        "near_dist"    "dpot"        
[41] "lat"          "lon"          "dbnd_sh"      "dbnd_sh2"     "dbnd_sh3"    
[46] "dbnd_sh4"     "x"            "y"            "x2"           "y2"          
[51] "xy"           "x3"           "y3"           "x2y"          "xy2"

In [5]:
mita_coef = function(threshold) {
    d = subset(mitaData, d_bnd < threshold)
    m = lm(lhhequiv ~
           pothuan_mita + x + y + x2 + y2 + xy + x3 + y3 + x2y + xy2 + 
           elv_sh + slope + infants + children + adults + 
           bfe4_1 + bfe4_2 + bfe4_3,
           data=d)
    c = cluster.vcov(m, d$district)
    coef = coeftest(m, c)['pothuan_mita',]
    pc = exp(coef[1]) - 1
    names(pc) = 'pct_change'
    return(c(coef, pc))
}

In [6]:
mita_coef(100)

Estimate Std. Error    t value   Pr(>|t|) pct_change 
-0.2841148  0.1988987 -1.4284395  0.1533794 -0.2473197

In [7]:
mita_coef(75)

Estimate Std. Error    t value   Pr(>|t|) pct_change 
-0.2164470  0.2074920 -1.0431582  0.2970958 -0.1946248

In [8]:
mita_coef(50)

Estimate Std. Error    t value   Pr(>|t|) pct_change 
-0.3310776  0.2192494 -1.5100502  0.1313484 -0.2818506

#### (i)
>What is the coefficient on mita? Are the results significant at the 5% level?
Interpret it.

**At thresholds of 100, 75, and 50, the coefficient on mita is -0.28, -0.22, and -0.33, respectively. It is not significant at the 5% level in any case (p = 0.15, 0.30, and 0.13 respectively). It means that the mita is associated with 20 to 28 percent decreases in equivalent household consumption (after exponentiating, controlling for all other covariates).**

### (c)

>Run the same regressions as in (b), but instead of polynomial terms in longitude and
latitude, use a cubic polynomial in distance to Potosi [dpot]. That is, include the
first, second and third powers of this variable in the regressions. Again, cluster the
standard errors by district and run the regression in 3 ways: first, for observations
where the distance to mita boundary [d-bnd] is less than 100km, next when it is less
than 75km, and lastly when it is less than 50km.

In [9]:
mitaData %<>% mutate(dpot2 = dpot^2,
                     dpot3 = dpot^3)

In [10]:
mita_coef_dpot = function(threshold) {
    d = subset(mitaData, d_bnd < threshold)
    m = lm(lhhequiv ~
           pothuan_mita + dpot + dpot2 + dpot3 +
           elv_sh + slope + infants + children + adults + 
           bfe4_1 + bfe4_2 + bfe4_3,
           data=d)
    c = cluster.vcov(m, d$district)
    coef = coeftest(m, c)['pothuan_mita',]
    pc = exp(coef[1]) - 1
    names(pc) = 'pct_change'
    return(c(coef, pc))
}

In [11]:
mita_coef_dpot(100)

Estimate   Std. Error      t value     Pr(>|t|)   pct_change 
-0.336809673  0.087002799 -3.871251020  0.000113041 -0.285955271

In [12]:
mita_coef_dpot(75)

Estimate   Std. Error      t value     Pr(>|t|)   pct_change 
-0.306956254  0.100648960 -3.049770759  0.002342649 -0.264317217

In [13]:
mita_coef_dpot(50)

Estimate    Std. Error       t value      Pr(>|t|)    pct_change 
-0.3286202730  0.0959422590 -3.4251879885  0.0006393381 -0.2800836635

#### (i)

>What is the coefficient on mita? Are the results significant at the 5% level?
Interpret it.

**The coefficient on `mita` ranges from -0.34 to -0.31 depending on the distance threshold, corresponding to reductions of equivalent household consumption between 26 and 29 percent. Each case is significant at the 5% level.**

### (d)

> Is there any difference between the coefficients on mita in (b) and (c) as well as their
significances? If yes, why might there be a difference? If no, why should we expect
the same?

**The `mita` coefficients are lower when using x-y polynomials farther from the border, compared to distance-to-Potosi polynomials. 

TODO****